# Andrew Benya

1. import csv file
2. loop over all URLs in csv file
- access website
- get relevant text
3. Compile all data into one corpus
4. tokenize the text
5. remove punctuation & numbers 
        (tests where I kept numbers & dates
           were less useful )
6. Lemmatize the text to gather related words
7. Find the most frequent 3-token phrases
8. Calculate the support of each frequent phrase
- Repeat for 2-token
- Repeat for 1-token
9. Aggreate data into DataFrame
10. Export Dataframe to csv

In [44]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import numpy as np

import math


In [45]:
def is_letter_only(word):
    for char in word:
        if not char.isalpha():
            return False
    return True

def is_alnum_only(word):
    for char in word:
        if not char.isalnum():
            return False
    return True


def truncate(number, decimals=0):
    """
    Returns a value truncated to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return math.trunc(number)

    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

In [46]:
headers = {
                "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
                "Accept-Encoding":"gzip, deflate, br",
                "Accept-Language":"en-US,en;q=0.9",
                "Connection":"keep-alive",
                "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.119 Safari/537.36",
                "Cache-Control":"max-age=0, no-cache, no-store",
                "Upgrade-Insecure-Requests": "1"
    }

# Hard coded file location here:

In [47]:
os.getcwd()
## HARD CODING WARNING
os.chdir('C:\\Users\\andre\\Documents\\BOWIE\\COSC 731')
os.getcwd()
df = pd.read_csv('Frequent Keywords.csv')
df.head()

,Data URLs,Frequent 1-Word Set,1-Word Support,Frequent 2-Word Set,2-Word Support,Frequent 3-Word Set,3-Word Support
0,https://solarsystem.nasa.gov/resources/1063/oc...,NaN,NaN,NaN,NaN,NaN,NaN
1,https://solarsystem.nasa.gov/resources/1064/oc...,NaN,NaN,NaN,NaN,NaN,NaN
2,https://solarsystem.nasa.gov/resources/1065/ma...,NaN,NaN,NaN,NaN,NaN,NaN
3,https://solarsystem.nasa.gov/resources/1066/de...,NaN,NaN,NaN,NaN,NaN,NaN
4,https://solarsystem.nasa.gov/resources/1067/fr...,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
weirdURL = 'https://solarsystem.nasa.gov/resources/1063/occator-crater-on-ceres-limb-short-exposure/'

## i had a tough time getting the request to work the first time.  
### all 546 other URLs connected on the first try, buit not the first one.
### i think there was some error in the URL or something, i had to hard code it to the same address


In [49]:
w = requests.get(weirdURL,headers=headers)
w.status_code

200

In [50]:
# the first URL was not the same as the URL from the brower
df['Data URLs'][0] == weirdURL

False

In [51]:

# So i just replaced it here, and it fixed the request
df.at[0,'Data URLs'] = weirdURL

In [52]:
corpus = []
URLS = []
index = 0

for item in df['Data URLs']:
    URLS.append(item)
    #print(item)
    
for URL in URLS:
    #print(URL)
    r = requests.get(URL,headers=headers)
    #print("URL # {}, Status Code:{}".format(index, r.status_code))
    if r.status_code == requests.codes.ok:
        soup = BeautifulSoup(r.content, "html.parser")
        results = soup.find_all(class_="content_page module")
        results[0].text.strip()
        document = results[0].text.replace("\n"," ")
        
    else:
        # this actually breaks the support calulation if 
        # you have any empty documents 
        # So all inputs need to work
        document = ' '
        
    corpus.append(document)
    # debugging index
    index += 1 
        
#print(len(corpus))

In [53]:
# filter out all non alpha characters
data_cleaned_only_alpha = []
for doc in corpus:
    doc_cleaned = ' '.join(word for word in doc.split()
                           if is_letter_only(word) )
    data_cleaned_only_alpha.append(doc_cleaned)
#print(data_cleaned_only_alpha[1])

In [54]:
lemmatizer = WordNetLemmatizer()

In [55]:
data_cleaned_only_alpha_lemmed = []
for doc in data_cleaned_only_alpha:
    doc = doc.lower()
    doc_cleaned = ' '.join(lemmatizer.lemmatize(word)
                           for word in doc.split()
                           if is_letter_only(word))
    data_cleaned_only_alpha_lemmed.append(doc_cleaned)
#print(data_cleaned_only_alpha_lemmed[1])

## lemmatizing takes the S off of the word "was" ...  hmmm

In [56]:
countVectorizer3token = CountVectorizer(ngram_range=(3,3),
                                 max_features=500,
                                 stop_words='english',
                                 lowercase=True)

In [57]:
three_token_fitted = countVectorizer3token.fit_transform(data_cleaned_only_alpha_lemmed)

In [58]:
feat_dict = sorted(countVectorizer3token.vocabulary_.keys())

In [59]:
three_token_fitted_array = three_token_fitted.toarray()

In [60]:
#Clear Lists

list_of_lists = []
frequent_set_3_grams = []
three_word_frequency = []
three_word_support = []
freq_word_indicies = []
for i in range(len(three_token_fitted_array)):
    frequent_set_3_grams.append([])
    three_word_frequency.append([])
    three_word_support.append([])
    freq_word_indicies.append([])
    list_of_lists.append([])
#print(frequent_set_3_grams)

In [61]:

for y in range(0,len(three_token_fitted_array)):
    for x in range(0,len(three_token_fitted_array[y])):
        if three_token_fitted_array[y][x] != 0:
            #print(y,x)
            freq_word_indicies[y].append(x)
            frequent_set_3_grams[y].append(feat_dict[x])
            three_word_frequency[y].append(three_token_fitted_array[y][x])
            
#print(len(frequent_set_3_grams))
#print(len(freq_word_indicies))
#print(len(three_word_frequency))

In [62]:

for x in range(0,len(three_word_frequency)):
    sum = 0
    for y in range(0,len(three_word_frequency[x])):
        sum += three_word_frequency[x][y]
        
    for y in range(0,len(three_word_frequency[x])): 
        #print(x,y)
        support = three_word_frequency[x][y] / sum
        three_word_support[x].append(truncate(support, 4))

        
#print(three_word_support)

In [63]:
countVectorizer2token = CountVectorizer(ngram_range=(2,2),
                                 max_features=500,
                                 stop_words='english',
                                 lowercase=True)

In [64]:
two_token_fitted = countVectorizer2token.fit_transform(data_cleaned_only_alpha_lemmed)
feat_dict_2_token = sorted(countVectorizer2token.vocabulary_.keys())
two_token_fitted_array = two_token_fitted.toarray()
#Clear Lists

list_of_lists = []
frequent_set_2_grams = []
two_word_frequency = []
two_word_support = []
freq_two_word_indicies = []
for i in range(len(two_token_fitted_array)):
    frequent_set_2_grams.append([])
    two_word_frequency.append([])
    two_word_support.append([])
    freq_two_word_indicies.append([])
    list_of_lists.append([])
#print(frequent_set_3_grams)

for y in range(0,len(two_token_fitted_array)):
    for x in range(0,len(two_token_fitted_array[y])):
        if two_token_fitted_array[y][x] != 0:
            #print(y,x)
            freq_two_word_indicies[y].append(x)
            frequent_set_2_grams[y].append(feat_dict_2_token[x])
            two_word_frequency[y].append(two_token_fitted_array[y][x])
            
#print(frequent_set_2_grams)
#print(len(freq_two_word_indicies))
#print(len(two_word_frequency))

for x in range(0,len(two_word_frequency)):
    sum = 0
    for y in range(0,len(two_word_frequency[x])):
        sum += two_word_frequency[x][y]
        
    for y in range(0,len(two_word_frequency[x])): 
        #print(x,y)
        support = two_word_frequency[x][y] / sum
        two_word_support[x].append(truncate(support, 4))

#print("two word support")        
#print(two_word_support)



In [65]:
countVectorizer1token = CountVectorizer(ngram_range=(1,1),
                                 max_features=500,
                                 stop_words='english',
                                 lowercase=True)

In [66]:
one_token_fitted = countVectorizer1token.fit_transform(data_cleaned_only_alpha_lemmed)
feat_dict_1_token = sorted(countVectorizer1token.vocabulary_.keys())
one_token_fitted_array = one_token_fitted.toarray()
#Clear Lists

list_of_lists = []
frequent_set_1_grams = []
one_word_frequency = []
one_word_support = []
freq_one_word_indicies = []
for i in range(len(one_token_fitted_array)):
    frequent_set_1_grams.append([])
    one_word_frequency.append([])
    one_word_support.append([])
    freq_one_word_indicies.append([])
    list_of_lists.append([])
#print(frequent_set_3_grams)

for y in range(0,len(one_token_fitted_array)):
    for x in range(0,len(one_token_fitted_array[y])):
        if one_token_fitted_array[y][x] != 0:
            #print(y,x)
            freq_one_word_indicies[y].append(x)
            frequent_set_1_grams[y].append(feat_dict_1_token[x])
            one_word_frequency[y].append(one_token_fitted_array[y][x])
            
#print(frequent_set_1_grams)
#print(len(freq_one_word_indicies))
#print(len(one_word_frequency))

for x in range(0,len(one_word_frequency)):
    sum = 0
    for y in range(0,len(one_word_frequency[x])):
        sum += one_word_frequency[x][y]
        
    for y in range(0,len(one_word_frequency[x])): 
        #print(x,y)
        support = one_word_frequency[x][y] / sum
        one_word_support[x].append(truncate(support, 4))

#print("one word support")        
#print(one_word_support)



In [67]:
data = {'DataURLS':df['Data URLs'],
       'frequent 1-word set': frequent_set_1_grams,
        '1-word support': one_word_support,
        'frequent 2-word set': frequent_set_2_grams,
        '2-word support': two_word_support,
        'frequent 3-word set': frequent_set_3_grams,
        '3-word support': three_word_support,
       }


newDF = pd.DataFrame(data)
newDF.head()

,DataURLS,frequent 1-word set,1-word support,frequent 2-word set,2-word support,frequent 3-word set,3-word support
0,https://solarsystem.nasa.gov/resources/1063/oc...,"[altitude, august, cere, crater, dawn, enlarge...","[0.04, 0.04, 0.08, 0.08, 0.04, 0.04, 0.08, 0.0...","[altitude mile, crater cere, dawn spacecraft, ...","[0.0666, 0.0666, 0.0666, 0.0666, 0.0666, 0.066...","[altitude mile enlarge, dawn spacecraft august...","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ..."
1,https://solarsystem.nasa.gov/resources/1064/oc...,"[altitude, august, cere, crater, dawn, enlarge...","[0.04, 0.04, 0.08, 0.08, 0.04, 0.04, 0.08, 0.0...","[altitude mile, crater cere, dawn spacecraft, ...","[0.0666, 0.0666, 0.0666, 0.0666, 0.0666, 0.066...","[altitude mile enlarge, dawn spacecraft august...","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ..."
2,https://solarsystem.nasa.gov/resources/1065/ma...,"[altitude, august, cere, crater, dawn, enlarge...","[0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.08, 0.0...","[altitude mile, crater rim, dawn spacecraft, e...","[0.0666, 0.0666, 0.0666, 0.0666, 0.1333, 0.066...","[altitude mile enlarge, dawn spacecraft august...","[0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.111..."
3,https://solarsystem.nasa.gov/resources/1066/de...,"[altitude, cere, crater, dawn, enlarge, fractu...","[0.05, 0.05, 0.1, 0.05, 0.05, 0.1, 0.05, 0.05,...","[altitude mile, crater september, crater wa, d...","[0.0588, 0.0588, 0.0588, 0.0588, 0.0588, 0.117...","[altitude mile enlarge, crater september image...","[0.0666, 0.0666, 0.0666, 0.0666, 0.0666, 0.133..."
4,https://solarsystem.nasa.gov/resources/1067/fr...,"[altitude, cere, crater, dawn, enlarge, floor,...","[0.0416, 0.0416, 0.0833, 0.0416, 0.0416, 0.083...","[altitude mile, crater september, crater wa, d...","[0.0476, 0.0476, 0.0476, 0.0476, 0.0476, 0.095...","[altitude mile enlarge, crater september image...","[0.0588, 0.0588, 0.0588, 0.0588, 0.0588, 0.117..."


In [68]:
newDF.to_csv('731projectANDREWBENYA.csv')